<a href="https://colab.research.google.com/github/jmanning21/31005_MachineLearning_Python/blob/master/Machine_Learning_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2: Practical Machine Learning Project
####Affnan Amir (13528841) and Julia Manning (12875795)

###Introduction
The algorithm chosen for this project is the ID3 Decision Tree building algorithm. In using this alrogithm, the data  

* Input = 
* Output = 

###Exploration

Challenges:
 - over-fitting the data

###Methodology

In [0]:
# Importing the required packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, metrics

In [0]:
# Importing the dataset from a url
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

# Read the csv and convert to a dataframe
d = pd.read_csv(url)
df = pd.DataFrame(data=d)

# See the first ten rows of the dataset
df.head(10)

In [46]:
# Cleaning some of the attributes: replace null values with median values
# Replacing NA values in age with the median age of each sex
median_values = df.groupby('Sex')['Age'].transform('median')
df['Age'].fillna(median_values, inplace=True)
df.head(6) 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,29.0,0,0,330877,8.4583,NaN,Q


In [0]:
# Check the information about the dataset
df.info()
# There are no missing values in the dataset

In [0]:
# Identify the target variable
data['Survived'],class_names = pd.factorize(data['Survived'])

print(class_names)
print(data['Survived'].unique())

# The algorithm requires the target variable to be in integer format, which it already is. 
# Therefore no data type changes need to be made.

In [0]:
# Add an additional column which may help with the accuracy of the algorithm
# Add a Child.Adult column based on the age of the individual
df['Child.Adult'] = 'Adult'

# Child = Age < 13 and Adult = Age >= 13 
df['Child.Adult'][df['Age'] < 13.0] = 'Child'

In [57]:
# Identifying the predictor variables and encode any string variables to their equivalent integer formats
df['Sex'],_ = pd.factorize(df['Sex'])
df['Embarked'],_ = pd.factorize(df['Embarked'])

df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Child.Adult
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,Adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,Adult
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,Adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,Adult
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,Adult
5,6,0,3,"Moran, Mr. James",0,29.0,0,0,330877,8.4583,NaN,2,Adult
6,7,0,1,"McCarthy, Mr. Timothy J",0,54.0,0,0,17463,51.8625,E46,0,Adult
7,8,0,3,"Palsson, Master. Gosta Leonard",0,2.0,3,1,349909,21.0750,NaN,0,Child
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",1,27.0,0,2,347742,11.1333,NaN,0,Adult
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",1,14.0,1,0,237736,30.0708,NaN,1,Adult


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null int64
dtypes: float64(2), int64(7), object(3)
memory usage: 83.6+ KB


###Evaluation
Improvements for future renditions: 
- When producing the ages for null values in the dataset, the ages could have been grouped by class and gender of the individual, rather than just using gender. This may produce a more accurate number. The age range for females is: 0.75-63, giving a range difference of 62.25, whilst the age range for males is: 0.42-80, giving a age range of 79.58. The age range is also specified further when Pclass is added.
- Use Gini Idex instead of Entropy?

###Conclusion

###Ethical Justification

- potential misuses of the technique
- utilitarian view?

###Video Pitch
*Attached to the final document*